In [1]:
import pycolmap
import os
import warnings
import h5py
import numpy as np
from tqdm import tqdm
from PIL import Image, ExifTags
from database import COLMAPDatabase, image_ids_to_pair_id

In [2]:
focal_lengths=[1962.48474121, 1951.8996582, 1961.0111084, 1959.28918457, 1950.74182129, 1958.7755127, 1958.4251709, 1944.15917969, 1946.36120605, 1941.74621582, 1939.5324707, 1939.76293945, 1938.9005127, 1948.53955078, 1949.68005371, 1938.31958008, 1939.8215332, 1948.44934082, 1947.72265625, 1941.68225098, 1941.9552002, 1950.00854492, 1946.20544434, 1953.12756348, 1954.95495605, 1946.65856934, 1946.92138672, 1947.0378418, 1957.24316406, 1955.84082031, 1955.99694824, 1964.03051758, 1963.94482422, 1962.08789063, 1963.74108887, 1958.48498535, 1956.71557617, 1956.8236084, 1957.41821289, 1957.71704102, 1951.01330566, 1957.50769043, 1951.16418457, 1961.11621094, 1952.26269531, 1963.64611816, 1963.93103027, 1954.46777344, 1955.40930176, 1963.29785156, 1956.07141113, 1965.29467773, 1957.90136719, 1966.39135742, 1972.14648438, 1973.1394043, 1972.38537598, 1965.23669434, 1967.86547852, 1966.42895508, 1972.11169434, 1951.40319824, 1953.61584473, 1958.88342285, 1961.48071289, 1962.52954102, 1963.95898438, 1965.29321289, 1956.17797852, 1957.82336426, 1957.7800293, 1959.79919434, 1962.64819336, 1962.56335449, 1961.63037109, 1969.87133789, 1966.9675293, 1970.76037598, 1973.61914063, 1972.86108398, 1970.65930176, 1956.65283203, 1957.38623047, 1970.7286377, 1973.71398926, 1963.85461426, 1956.2199707, 1955.91796875, 1955.30969238, 1954.93432617, 1963.49963379, 1962.13647461, 1963.15100098, 1970.89355469, 1969.96008301, 1972.24621582, 1969.15478516, 1969.48339844, 1970.30224609, 1956.43164063]

In [18]:
def add_keypoints(db, h5_path, image_path):
    keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')
    first_image_path = os.path.join(image_path, next(iter(keypoint_f.keys())))
    image = Image.open(first_image_path)
    width, height = image.size
    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]

        fname_without_ext = os.path.splitext(filename)[0]
        path = os.path.join(image_path, filename)
        if not os.path.isfile(path):
            raise IOError(f'Invalid image path {path}')

        param_arr = np.array([focal_lengths[int(fname_without_ext)], width / 2, height / 2])
        camera_id = db.add_camera(0, width, height, param_arr)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id

        db.add_keypoints(image_id, keypoints)

    return fname_to_id

In [9]:
def add_matches(db, h5_path, fname_to_id):
    match_file = h5py.File(os.path.join(h5_path, 'matches.h5'), 'r')
    
    added = set()
    n_keys = len(match_file.keys())
    n_total = (n_keys * (n_keys - 1)) // 2

    with tqdm(total=n_total) as pbar:
        for key_1 in match_file.keys():
            group = match_file[key_1]
            for key_2 in group.keys():
                id_1 = fname_to_id[key_1]
                id_2 = fname_to_id[key_2]

                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                    continue
            
                matches = group[key_2][()]
                db.add_matches(id_1, id_2, matches)

                added.add(pair_id)

                pbar.update(1)

In [20]:
def import_into_colmap(img_dir,
                       feature_dir ='features',
                       database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )

    db.commit()
    return

In [21]:
feature_dir = 'features'
images = 'images'
output_path = 'Reconstruction3D'
database_path = 'colmap_loftr.db'

In [25]:
if os.path.exists(database_path):
    os.remove(database_path)

In [26]:
import_into_colmap(images,feature_dir=feature_dir, database_path=database_path)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 308.22it/s]


Total keys: 98
Total unique pairs expected: 4753


  2%|▏         | 79/4753 [00:00<00:06, 719.97it/s]

Processing pair: (0.jpg, 5.jpg) with IDs (1, 46)
Processing pair: (0.jpg, 60.jpg) with IDs (1, 58)
Processing pair: (1.jpg, 10.jpg) with IDs (2, 3)
Processing pair: (1.jpg, 11.jpg) with IDs (2, 4)
Processing pair: (1.jpg, 12.jpg) with IDs (2, 5)
Processing pair: (1.jpg, 13.jpg) with IDs (2, 6)
Processing pair: (1.jpg, 17.jpg) with IDs (2, 10)
Processing pair: (1.jpg, 19.jpg) with IDs (2, 12)
Processing pair: (1.jpg, 21.jpg) with IDs (2, 15)
Processing pair: (1.jpg, 25.jpg) with IDs (2, 19)
Processing pair: (1.jpg, 28.jpg) with IDs (2, 22)
Processing pair: (1.jpg, 30.jpg) with IDs (2, 25)
Processing pair: (1.jpg, 31.jpg) with IDs (2, 26)
Processing pair: (1.jpg, 33.jpg) with IDs (2, 28)
Processing pair: (1.jpg, 34.jpg) with IDs (2, 29)
Processing pair: (1.jpg, 35.jpg) with IDs (2, 30)
Processing pair: (1.jpg, 36.jpg) with IDs (2, 31)
Processing pair: (1.jpg, 37.jpg) with IDs (2, 32)
Processing pair: (1.jpg, 38.jpg) with IDs (2, 33)
Processing pair: (1.jpg, 39.jpg) with IDs (2, 34)
Proce

...

Processing pair: (78.jpg, 13.jpg) with IDs (77, 6)
Processing pair: (78.jpg, 17.jpg) with IDs (77, 10)
Processing pair: (78.jpg, 19.jpg) with IDs (77, 12)
Processing pair: (78.jpg, 2.jpg) with IDs (77, 13)
Processing pair: (78.jpg, 21.jpg) with IDs (77, 15)
Processing pair: (78.jpg, 25.jpg) with IDs (77, 19)
Processing pair: (78.jpg, 28.jpg) with IDs (77, 22)
Processing pair: (78.jpg, 3.jpg) with IDs (77, 24)
Processing pair: (78.jpg, 30.jpg) with IDs (77, 25)
Processing pair: (78.jpg, 31.jpg) with IDs (77, 26)
Processing pair: (78.jpg, 33.jpg) with IDs (77, 28)
Processing pair: (78.jpg, 34.jpg) with IDs (77, 29)
Processing pair: (78.jpg, 35.jpg) with IDs (77, 30)
Processing pair: (78.jpg, 36.jpg) with IDs (77, 31)
Processing pair: (78.jpg, 37.jpg) with IDs (77, 32)
Processing pair: (78.jpg, 38.jpg) with IDs (77, 33)
Processing pair: (78.jpg, 39.jpg) with IDs (77, 34)
Processing pair: (78.jpg, 40.jpg) with IDs (77, 36)
Processing pair: (78.jpg, 41.jpg) with IDs (77, 37)
Processing pair

 52%|█████▏    | 2474/4753 [00:02<00:01, 1165.48it/s]

Processing pair: (82.jpg, 68.jpg) with IDs (82, 66)
Processing pair: (82.jpg, 7.jpg) with IDs (82, 68)
Processing pair: (82.jpg, 70.jpg) with IDs (82, 69)
Processing pair: (82.jpg, 72.jpg) with IDs (82, 71)
Processing pair: (82.jpg, 75.jpg) with IDs (82, 74)
Processing pair: (82.jpg, 80.jpg) with IDs (82, 80)
Processing pair: (82.jpg, 84.jpg) with IDs (82, 84)
Processing pair: (82.jpg, 87.jpg) with IDs (82, 87)
Processing pair: (82.jpg, 89.jpg) with IDs (82, 89)
Processing pair: (82.jpg, 90.jpg) with IDs (82, 91)
Processing pair: (82.jpg, 95.jpg) with IDs (82, 96)
Processing pair: (82.jpg, 96.jpg) with IDs (82, 97)
Processing pair: (82.jpg, 98.jpg) with IDs (82, 99)
Processing pair: (82.jpg, 99.jpg) with IDs (82, 100)
Processing pair: (83.jpg, 10.jpg) with IDs (83, 3)
Processing pair: (83.jpg, 11.jpg) with IDs (83, 4)
Processing pair: (83.jpg, 12.jpg) with IDs (83, 5)
Processing pair: (83.jpg, 13.jpg) with IDs (83, 6)
Processing pair: (83.jpg, 17.jpg) with IDs (83, 10)
Processing pair:

 55%|█████▍    | 2608/4753 [00:02<00:02, 905.45it/s] 


Processing pair: (93.jpg, 15.jpg) with IDs (94, 8)
Processing pair: (93.jpg, 23.jpg) with IDs (94, 17)
Processing pair: (93.jpg, 4.jpg) with IDs (94, 35)
Processing pair: (93.jpg, 42.jpg) with IDs (94, 38)
Processing pair: (93.jpg, 43.jpg) with IDs (94, 39)
Processing pair: (93.jpg, 45.jpg) with IDs (94, 41)
Processing pair: (93.jpg, 48.jpg) with IDs (94, 44)
Processing pair: (93.jpg, 5.jpg) with IDs (94, 46)
Processing pair: (93.jpg, 54.jpg) with IDs (94, 51)
Processing pair: (93.jpg, 68.jpg) with IDs (94, 66)
Processing pair: (93.jpg, 69.jpg) with IDs (94, 67)
Processing pair: (93.jpg, 7.jpg) with IDs (94, 68)
Processing pair: (93.jpg, 76.jpg) with IDs (94, 75)
Processing pair: (93.jpg, 77.jpg) with IDs (94, 76)
Processing pair: (93.jpg, 80.jpg) with IDs (94, 80)
Processing pair: (93.jpg, 85.jpg) with IDs (94, 85)
Processing pair: (93.jpg, 89.jpg) with IDs (94, 89)
Processing pair: (93.jpg, 9.jpg) with IDs (94, 90)
Processing pair: (93.jpg, 95.jpg) with IDs (94, 96)
Processing pair: 

In [9]:
if not os.path.isdir(output_path):
    os.makedirs(output_path)

In [10]:
# This does RANSAC
pycolmap.match_exhaustive(database_path)

I20240603 10:59:03.906268 1658114 misc.cc:198] 
Exhaustive feature matching
I20240603 10:59:03.947798 1658114 feature_matching.cc:231] Matching block [1/2, 1/2]
I20240603 11:00:31.630089 1658114 feature_matching.cc:46]  in 87.680s
I20240603 11:00:31.653913 1658114 feature_matching.cc:231] Matching block [1/2, 2/2]
I20240603 11:02:08.834997 1658114 feature_matching.cc:46]  in 97.179s
I20240603 11:02:08.879092 1658114 feature_matching.cc:231] Matching block [2/2, 1/2]
I20240603 11:03:44.291188 1658114 feature_matching.cc:46]  in 95.410s
I20240603 11:03:44.314370 1658114 feature_matching.cc:231] Matching block [2/2, 2/2]
I20240603 11:04:46.037727 1658114 feature_matching.cc:46]  in 61.722s
I20240603 11:04:46.130093 1658114 timer.cc:91] Elapsed time: 5.704 [minutes]


In [11]:
maps = pycolmap.incremental_mapping(database_path, images, output_path)
maps[0].write(output_path)

I20240603 11:04:46.802704 1662539 misc.cc:198] 
Loading database
I20240603 11:04:46.809837 1662539 database_cache.cc:54] Loading cameras...
I20240603 11:04:46.809986 1662539 database_cache.cc:64]  100 in 0.000s
I20240603 11:04:46.810012 1662539 database_cache.cc:72] Loading matches...
I20240603 11:04:46.842751 1662539 database_cache.cc:78]  2608 in 0.033s
I20240603 11:04:46.842794 1662539 database_cache.cc:94] Loading images...
I20240603 11:04:46.899914 1662539 database_cache.cc:143]  100 in 0.057s (connected 100)
I20240603 11:04:46.900079 1662539 database_cache.cc:154] Building correspondence graph...
I20240603 11:04:47.368252 1662539 database_cache.cc:190]  in 0.468s (ignored 0)
I20240603 11:04:47.370640 1662539 timer.cc:91] Elapsed time: 0.009 [minutes]
I20240603 11:04:47.393347 1662539 misc.cc:198] 
Finding good initial image pair
I20240603 11:04:47.788074 1662539 misc.cc:198] 
Initializing with image pair #33 and #78
I20240603 11:04:47.790149 1662539 misc.cc:198] 
Global bundle ad

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  8.049800e+03    0.00e+00    4.47e+05   0.00e+00   0.00e+00  1.00e+04        0    2.23e-03    1.60e-02
   1  1.030242e+03    7.02e+03    8.81e+04   9.77e+01   9.01e-01  2.06e+04        1    5.23e-03    2.12e-02
   2  2.339700e+02    7.96e+02    2.77e+03   5.03e+01   9.96e-01  6.18e+04        1    1.30e-03    2.26e-02
   3  2.271292e+02    6.84e+00    1.34e+04   4.05e+01   5.97e-01  6.23e+04        1    1.40e-03    2.40e-02
   4  2.206674e+02    6.46e+00    3.94e+03   2.04e+01   9.22e-01  1.55e+05        1    9.85e-04    2.50e-02
   5  2.193015e+02    1.37e+00    3.40e+03   1.99e+01   7.09e-01  1.68e+05        1    9.50e-04    2.60e-02
   6  2.186036e+02    6.98e-01    4.53e+02   6.99e+00   9.79e-01  5.03e+05        1    1.14e-03    2.71e-02
   7  2.185644e+02    3.92e-02    2.33e+01   3.85e+00   9.11e-01  1.13e+06        1    1.10e-03    2.83e-02
   8  2.185697e+02   -5.40e-

I20240603 11:04:48.126802 1662539 misc.cc:198] 
Registering image #12 (3)
I20240603 11:04:48.126849 1662539 incremental_mapper.cc:495] => Image sees 158 / 8247 points
I20240603 11:04:48.336334 1662539 misc.cc:205] 
Pose refinement report
----------------------
I20240603 11:04:48.336536 1662539 bundle_adjustment.cc:942] 
    Residuals : 296
   Parameters : 7
   Iterations : 15
         Time : 0.00468206 [s]
 Initial cost : 0.84548 [px]
   Final cost : 0.79231 [px]
  Termination : Convergence

I20240603 11:04:48.337246 1662539 incremental_mapper.cc:40] => Continued observations: 148
I20240603 11:04:48.357221 1662539 incremental_mapper.cc:43] => Added observations: 1408
I20240603 11:04:48.483666 1662539 misc.cc:205] 
Bundle adjustment report
------------------------
I20240603 11:04:48.483711 1662539 bundle_adjustment.cc:942] 
    Residuals : 3964
   Parameters : 2759
   Iterations : 22
         Time : 0.120982 [s]
 Initial cost : 0.673331 [px]
   Final cost : 0.533559 [px]
  Termination :

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.470496e+03    0.00e+00    6.11e+02   0.00e+00   0.00e+00  1.00e+04        0    1.60e-03    1.11e-02
   1  1.454266e+03    1.62e+01    5.45e+00   1.72e+00   1.00e+00  3.00e+04        1    4.96e-03    1.61e-02
   2  1.454108e+03    1.58e-01    4.67e+00   1.26e+00   9.90e-01  9.00e+04        1    6.47e-03    2.26e-02
   3  1.454057e+03    5.06e-02    1.66e+01   1.34e+00   9.92e-01  2.70e+05        1    5.04e-03    2.82e-02
   4  1.453991e+03    6.64e-02    1.26e+02   3.02e+00   9.07e-01  5.88e+05        1    5.23e-03    3.34e-02
   5  1.453934e+03    5.61e-02    2.75e+02   4.44e+00   6.42e-01  6.01e+05        1    6.58e-03    4.00e-02
   6  1.453872e+03    6.26e-02    1.31e+02   3.05e+00   9.03e-01  1.26e+06        1    6.31e-03    4.64e-02
   7  1.453849e+03    2.28e-02    1.43e+02   3.19e+00   7.35e-01  1.41e+06        1    5.24e-03    5.16e-02
   8  1.453835e+03    1.43e-

I20240603 11:04:48.613040 1662539 misc.cc:205] 
Bundle adjustment report
------------------------
I20240603 11:04:48.613097 1662539 bundle_adjustment.cc:942] 
    Residuals : 3970
   Parameters : 2753
   Iterations : 14
         Time : 0.0805681 [s]
 Initial cost : 0.608607 [px]
   Final cost : 0.605148 [px]
  Termination : Convergence

I20240603 11:04:48.617569 1662539 incremental_mapper.cc:175] => Completed observations: 2
I20240603 11:04:48.618927 1662539 incremental_mapper.cc:178] => Merged observations: 0
I20240603 11:04:48.620322 1662539 incremental_mapper.cc:160] => Filtered observations: 0
I20240603 11:04:48.620398 1662539 incremental_mapper.cc:119] => Changed observations: 0.001008
I20240603 11:04:48.620451 1662539 misc.cc:198] 
Global bundle adjustment
I20240603 11:04:48.685343 1662539 misc.cc:205] 
Bundle adjustment report
------------------------
I20240603 11:04:48.685392 1662539 bundle_adjustment.cc:942] 
    Residuals : 3974
   Parameters : 2753
   Iterations : 10
       

...

I20240603 11:21:14.004061 1662539 misc.cc:205] 
Bundle adjustment report
------------------------
I20240603 11:21:14.005147 1662539 bundle_adjustment.cc:942] 
    Residuals : 370312
   Parameters : 127346
   Iterations : 17
         Time : 11.0898 [s]
 Initial cost : 0.843535 [px]
   Final cost : 0.838227 [px]
  Termination : Convergence

I20240603 11:21:14.428400 1662539 incremental_mapper.cc:175] => Completed observations: 453
I20240603 11:21:14.512557 1662539 incremental_mapper.cc:178] => Merged observations: 31
I20240603 11:21:14.580494 1662539 incremental_mapper.cc:160] => Filtered observations: 93
I20240603 11:21:14.580540 1662539 incremental_mapper.cc:119] => Changed observations: 0.003116
I20240603 11:21:14.580569 1662539 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.621576e+05    0.00e+00    2.74e+04   0.00e+00   0.00e+00  1.00e+04        0    1.42e-01    1.85e+00
   1  2.610053e+05    1.15e+03    5.53e+03   1.68e+01   1.00e+00  3.00e+04        1    1.09e+00    2.93e+00
   2  2.609835e+05    2.18e+01    2.64e+03   1.10e+01   1.01e+00  9.00e+04        1    7.08e-01    3.64e+00
   3  2.609709e+05    1.27e+01    5.34e+03   8.55e+00   9.62e-01  2.70e+05        1    5.28e-01    4.17e+00
   4  2.609658e+05    5.01e+00    2.40e+03   5.80e+00   1.04e+00  8.10e+05        1    5.65e-01    4.74e+00
   5  2.609650e+05    8.42e-01    2.90e+02   2.67e+00   1.13e+00  2.43e+06        1    5.36e-01    5.27e+00
   6  2.609649e+05    6.05e-02    2.13e+01   1.11e+00   1.27e+00  7.29e+06        1    5.51e-01    5.82e+00
   7  2.609649e+05    5.97e-03    1.80e+00   4.37e-01   1.34e+00  2.19e+07        1    5.66e-01    6.39e+00
   8  2.609649e+05    7.57e-

I20240603 11:21:23.356696 1662539 misc.cc:205] 
Bundle adjustment report
------------------------
I20240603 11:21:23.356756 1662539 bundle_adjustment.cc:942] 
    Residuals : 371032
   Parameters : 127298
   Iterations : 11
         Time : 8.08669 [s]
 Initial cost : 0.840573 [px]
   Final cost : 0.838659 [px]
  Termination : Convergence

I20240603 11:21:23.795819 1662539 incremental_mapper.cc:175] => Completed observations: 202
I20240603 11:21:23.877772 1662539 incremental_mapper.cc:178] => Merged observations: 27
I20240603 11:21:23.945161 1662539 incremental_mapper.cc:160] => Filtered observations: 57
I20240603 11:21:23.945206 1662539 incremental_mapper.cc:119] => Changed observations: 0.001542
I20240603 11:21:23.945235 1662539 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.618455e+05    0.00e+00    9.47e+03   0.00e+00   0.00e+00  1.00e+04        0    1.43e-01    1.84e+00
   1  2.613148e+05    5.31e+02    1.33e+03   9.96e+00   1.00e+00  3.00e+04        1    1.02e+00    2.86e+00
   2  2.613069e+05    7.90e+00    1.25e+03   8.76e+00   1.01e+00  9.00e+04        1    6.20e-01    3.48e+00
   3  2.613021e+05    4.87e+00    2.25e+03   8.53e+00   1.01e+00  2.70e+05        1    5.22e-01    4.00e+00
   4  2.613005e+05    1.59e+00    9.07e+02   5.32e+00   1.09e+00  8.10e+05        1    5.31e-01    4.53e+00
   5  2.613003e+05    2.15e-01    9.39e+01   2.67e+00   1.19e+00  2.43e+06        1    5.31e-01    5.07e+00
   6  2.613003e+05    1.94e-02    8.24e+00   1.24e+00   1.29e+00  7.29e+06        1    5.42e-01    5.61e+00
   7  2.613002e+05    2.14e-03    1.78e+00   5.20e-01   1.35e+00  2.19e+07        1    5.29e-01    6.14e+00
   8  2.613002e+05    2.82e-

I20240603 11:21:31.876641 1662539 misc.cc:205] 
Bundle adjustment report
------------------------
I20240603 11:21:31.876685 1662539 bundle_adjustment.cc:942] 
    Residuals : 371322
   Parameters : 127271
   Iterations : 10
         Time : 7.22548 [s]
 Initial cost : 0.839745 [px]
   Final cost : 0.83887 [px]
  Termination : Convergence

I20240603 11:21:32.330502 1662539 incremental_mapper.cc:175] => Completed observations: 127
I20240603 11:21:32.410989 1662539 incremental_mapper.cc:178] => Merged observations: 23
I20240603 11:21:32.478997 1662539 incremental_mapper.cc:160] => Filtered observations: 34
I20240603 11:21:32.479044 1662539 incremental_mapper.cc:119] => Changed observations: 0.000991
I20240603 11:21:32.479091 1662539 incremental_mapper.cc:167] => Filtered images: 0
I20240603 11:21:32.510224 1662539 timer.cc:91] Elapsed time: 16.761 [minutes]


In [12]:
!colmap gui

qt.qpa.fonts: Populating font family aliases took 200 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 
QOpenGLVertexArrayObject::destroy() failed to restore current context
